In [1]:
import os
os.environ['PYPROJ_GLOBAL_CONTEXT'] = 'ON'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../")
import regionate as reg
import sectionate as sec

In [4]:
import geopandas as gpd
from shapely.geometry import Polygon
import regionmask

In [5]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
div_cmap = plt.get_cmap("RdBu_r").copy()
div_cmap.set_bad("grey")

cmap = plt.get_cmap("viridis").copy()
cmap.set_bad("grey")

#### Helper functions

In [6]:
def basin_boundary_grid_indices(basin, ocean_grid):
    corner_i, corner_j, corner_lons, corner_lats = sec.create_section_composite(
        ocean_grid['geolon_c'],
        ocean_grid['geolat_c'],
        np.append(basin.lons, basin.lons[0]),
        np.append(basin.lats, basin.lats[0]),
        closed=True
    )
    
    return corner_i, corner_j, corner_lons, corner_lats

In [7]:
def basin_interior_grid_mask(basin, ocean_grid):
    polygon_geom = Polygon(zip(np.mod(basin.corner_lons, 360), basin.corner_lats))

    crs = 'epsg:4326'
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
    basin_grid_mask = ~np.isnan(regionmask.mask_geopandas(polygon, np.mod(ocean_grid['geolon'], 360), lat=ocean_grid['geolat'], wrap_lon="360"))
    
    return basin_grid_mask

In [8]:
def conform_basin_to_ocean_grid(basin, ocean_grid):
    basin.corner_i, basin.corner_j, basin.corner_lons, basin.corner_lats = basin_boundary_grid_indices(basin, ocean_grid)
    basin.mask = basin_interior_grid_mask(basin, ocean_grid)

### Load connected regions

In [9]:
import pickle
BasinsFile = "pickled_regions/PJ2010_regions"
with open(BasinsFile, 'rb') as pickle_file:
    regions = pickle.load(pickle_file)

### Load processed CMp125 output

In [10]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    
    # get correct geo coords from super grid
    sg = xr.open_dataset("/archive/Raphael.Dussin/datasets/OM4p125/mosaic_c192_om4p125_bedmachine_v20210310_hydrographyKDunne20210614_unpacked/ocean_hgrid.nc")
    zdiag_path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/"
    ocean_grid = xr.open_dataset(f"{zdiag_path}ocean_annual_z.static.nc")
    ocean_grid = ocean_grid.assign_coords({'geolon'  : xr.DataArray(sg['x'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolat'  : xr.DataArray(sg['y'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolon_u': xr.DataArray(sg['x'][1::2,0::2].data, dims=["yh", "xq"]),
                                           'geolat_u': xr.DataArray(sg['y'][1::2,0::2].data, dims=["yh", "xq"]),
                                           'geolon_v': xr.DataArray(sg['x'][0::2,1::2].data, dims=["yq", "xh"]),
                                           'geolat_v': xr.DataArray(sg['y'][0::2,1::2].data, dims=["yq", "xh"]),
                                           'geolon_c': xr.DataArray(sg['x'][0::2,0::2].data, dims=["yq", "xq"]),
                                           'geolat_c': xr.DataArray(sg['y'][0::2,0::2].data, dims=["yq", "xq"])})

In [11]:
heat = xr.open_dataset("/work/hfd/CM4-highres-WMT/data/abyssal_heat_content_annual.nc").isel(time=-1)

### Conform basin boundaries to model grid

In [ ]:
print("Conforming regions to ocean grid: ", end="")
for (name, basin) in regions.Basins.items():
    print(name, end=", ")
    conform_basin_to_ocean_grid(regions.Basins[name], ocean_grid)

Conforming regions to ocean grid: 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 2, 30, 31, 32, 33, 3, 4, 5, 6, 7, 8, 

In [ ]:
import pickle
BasinsFile = "pickled_regions/PJ2010_regions_gridded"
with open(BasinsFile, 'wb') as pickle_file:
    pickle.dump(regions, pickle_file)

### Verfication plots of region boundaries and interior masks

In [ ]:
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
data_crs = ccrs.PlateCarree()
subplot_kws=dict(projection=data_crs, facecolor='grey')

In [ ]:
basin.corner_lons

In [ ]:
for bname in ["1", "2", "3", "4"]:
    basin = regions.Basins[bname]
    fig = plt.figure(figsize=(14, 10))
    ax = plt.axes(**subplot_kws)
    pc = ax.pcolormesh(ocean_grid['geolon_c'], ocean_grid['geolat_c'], basin.mask, transform=data_crs, cmap="Greys", vmin=0, vmax=2)
    ax.plot(basin.lons, basin.lats, "C1x", transform=data_crs, markersize=4)
    ax.plot(basin.corner_lons, basin.corner_lats, "k.-", lw=0.5, markersize=3, transform=data_crs)

In [ ]:
for bname in ["1", "2", "3", "4"]:
    basin = regions.Basins[bname]
    fig = plt.figure(figsize=(14, 10))
    ax = plt.axes()
    ax.plot(basin.lons, basin.lats, "C1x", markersize=4)
    ax.plot(np.mod(basin.corner_lons, 360), basin.corner_lats, "k.-", lw=0.5, markersize=3)

In [ ]:
corner_i_ref, corner_j_ref, corner_lons_ref, corner_lats_ref = sec.create_section_composite(
    ocean_grid['geolon_c'],
    ocean_grid['geolat_c'],
    np.append(basin.lons[::10], basin.lons[0]),
    np.append(basin.lats[::10], basin.lats[0]),
    closed=True
)

In [ ]:
lon_offset = np.mod( np.min(basin.lons[::10]) + 300., 360.)-300. -5.
lon_offset

In [ ]:
roll_idx = (np.abs(ocean_grid['geolon_c'].min("yq") - lon_offset)).argmin('xq').values

In [ ]:
tmp = ocean_grid.roll({'xq': -roll_idx, 'xh': -roll_idx}, roll_coords=True)

In [ ]:
tmp = tmp.assign_coords({k:np.mod(v-lon_offset, 360.)+lon_offset for (k,v) in tmp.coords.items() if ('x' in k)})
tmp = tmp.assign_coords({k:np.mod(v-lon_offset, 360.)+lon_offset for (k,v) in tmp.coords.items() if ('lon' in k)})

In [ ]:
corner_i, corner_j, corner_lons, corner_lats = sec.create_section_composite(
    tmp['geolon_c'],
    tmp['geolat_c'],
    np.append(basin.lons[::10], basin.lons[0]),
    np.append(basin.lats[::10], basin.lats[0]),
    closed=True
)

In [ ]:
corner_i

In [ ]:
plt.plot(basin.lons[::10], basin.lats[::10], "kx", markersize=5, label="segment vertices")
plt.plot(corner_lons_ref, corner_lats_ref, ".-", markersize=1, lw=3, alpha=0.5, label="current implementation")
plt.plot(corner_lons, corner_lats, ".-", markersize=1, lw=0.5, label="shortest wrapping")
plt.axvline(60, linestyle="--", color="k")
plt.legend()